In [260]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
from sklearn.metrics import mutual_info_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer

In [166]:
url = "https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv"

In [167]:
df = pd.read_csv("data-week-3.csv")
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


# Data preparation

In [168]:
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [169]:
categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)
categorical_columns

['lead_source', 'industry', 'employment_status', 'location']

In [170]:
numerical_columns = df.select_dtypes(include='number').columns.tolist()
numerical_columns.remove('converted')
numerical_columns

['number_of_courses_viewed',
 'annual_income',
 'interaction_count',
 'lead_score']

In [171]:
for c in categorical_columns:
    df[c] = df[c].str.lower()
    df[c] = df[c].fillna('NA')

In [172]:
df.isnull().sum()

lead_source                   0
industry                      0
number_of_courses_viewed      0
annual_income               181
employment_status             0
location                      0
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [173]:
for c in numerical_columns:
    df[c] = df[c].fillna(0.0)

In [174]:
df.isnull().sum()

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

# Q1 - What is the most frequent observation (mode) for the column industry?

In [175]:
df['industry'].value_counts()

industry
retail           203
finance          200
other            198
healthcare       187
education        187
technology       179
manufacturing    174
NA               134
Name: count, dtype: int64

In [176]:
df['industry'].mode()

0    retail
Name: industry, dtype: object

# Q2 - Biggest correlation

In [237]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

In [238]:
len(df_train), len(df_val), len(df_test)

(876, 293, 293)

In [239]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [240]:
y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

del df_train['converted']
del df_val['converted']
del df_test['converted']

In [241]:
df_full_train[numerical_columns].corrwith(df_full_train.converted).abs()

number_of_courses_viewed    0.442068
annual_income               0.029612
interaction_count           0.378482
lead_score                  0.225641
dtype: float64

In [242]:
correlation_matrix = df[numerical_columns].corr()
print(correlation_matrix)

                          number_of_courses_viewed  annual_income  \
number_of_courses_viewed                  1.000000       0.009770   
annual_income                             0.009770       1.000000   
interaction_count                        -0.023565       0.027036   
lead_score                               -0.004879       0.015610   

                          interaction_count  lead_score  
number_of_courses_viewed          -0.023565   -0.004879  
annual_income                      0.027036    0.015610  
interaction_count                  1.000000    0.009888  
lead_score                         0.009888    1.000000  


# Q3 - Biggest MI

In [243]:
categorical_columns

['lead_source', 'industry', 'employment_status', 'location']

In [244]:
round(mutual_info_score(df_full_train.converted, df_full_train.lead_source), 2)

0.03

In [245]:
round(mutual_info_score(df_full_train.converted, df_full_train.industry), 2)

0.01

In [246]:
round(mutual_info_score(df_full_train.converted, df_full_train.employment_status), 2)

0.01

In [247]:
round(mutual_info_score(df_full_train.converted, df_full_train.location), 2)

0.0

# Q4 - Accuracy

### One-hot encoding

In [248]:
dv = DictVectorizer(sparse=False)

train_dict = df_train[categorical_columns + numerical_columns].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical_columns + numerical_columns].to_dict(orient='records')
X_val = dv.transform(val_dict)

test_dict = df_test[categorical_columns + numerical_columns].to_dict(orient='records')
X_test = dv.transform(test_dict)

### Training logistic regression

In [249]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'liblinear'
,max_iter,1000
,multi_class,'deprecated'


In [250]:
model.intercept_[0]

np.float64(-0.06914728027832363)

In [251]:
model.coef_[0].round(3)

array([-0.   , -0.015,  0.034,  0.003,  0.012, -0.103, -0.025,  0.049,
       -0.02 , -0.013, -0.003, -0.009, -0.032, -0.016,  0.311,  0.051,
        0.02 , -0.012, -0.012, -0.115,  0.08 , -0.03 ,  0.004, -0.011,
       -0.011, -0.006,  0.008,  0.006, -0.033, -0.025,  0.454])

In [252]:
y_pred = model.predict_proba(X_val)[:, 1]

In [253]:
converted_decision = (y_pred >= 0.5)

In [254]:
round((y_val == converted_decision).mean(), 2)

np.float64(0.7)

In [255]:
df_pred = pd.DataFrame()
df_pred['probability'] = y_pred
df_pred['prediction'] = converted_decision.astype(int)
df_pred['actual'] = y_val

In [256]:
df_pred['correct'] = df_pred.prediction == df_pred.actual

In [265]:
accuracy_base = df_pred.correct.mean()
print(f"Accuracy: {round(accuracy_base, 2)}")

Accuracy: 0.7


# Q5 - Feature selection

In [267]:
dv = DictVectorizer(sparse=False)

all_columns = categorical_columns + numerical_columns

train_dict = df_train[categorical_columns + numerical_columns].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical_columns + numerical_columns].to_dict(orient='records')
X_val = dv.transform(val_dict)

test_dict = df_test[categorical_columns + numerical_columns].to_dict(orient='records')
X_test = dv.transform(test_dict)

model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_val)

accuracy_feature = accuracy_score(y_val, y_pred)

for c in ['industry', 'employment_status', 'lead_score']:

    all_columns = categorical_columns + numerical_columns
    all_columns.remove(c)
    
    train_dict = df_train[all_columns].to_dict(orient='records')
    X_train = dv.fit_transform(train_dict)
    
    val_dict = df_val[all_columns].to_dict(orient='records')
    X_val = dv.transform(val_dict)
    
    test_dict = df_test[all_columns].to_dict(orient='records')
    X_test = dv.transform(test_dict)
    
    model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_val)
    
    accuracy_remove_feature = accuracy_score(y_val, y_pred)
    print(f"Removing feature {c} accuracy: {accuracy_remove_feature}")
    print(f"Difference from base accuracy: {accuracy_base - accuracy_remove_feature}\n")

Removing feature industry accuracy: 0.6996587030716723
Difference from base accuracy: 0.0

Removing feature employment_status accuracy: 0.6962457337883959
Difference from base accuracy: 0.0034129692832763903

Removing feature lead_score accuracy: 0.7064846416382252
Difference from base accuracy: -0.0068259385665528916



# Q6 - Parameter tuning

In [270]:
C = [0.01, 0.1, 1, 10, 100]

dv = DictVectorizer(sparse=False)

all_columns = categorical_columns + numerical_columns

train_dict = df_train[all_columns].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[all_columns].to_dict(orient='records')
X_val = dv.transform(val_dict)

test_dict = df_test[all_columns].to_dict(orient='records')
X_test = dv.transform(test_dict)

for c in C:
        
    model = LogisticRegression(solver='liblinear', C=c, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_val)
    
    accuracy_regularization = accuracy_score(y_val, y_pred)
    print(f"Regularization value {c} accuracy: {accuracy_regularization:.3f}")

Regularization value 0.01 accuracy: 0.700
Regularization value 0.1 accuracy: 0.700
Regularization value 1 accuracy: 0.700
Regularization value 10 accuracy: 0.700
Regularization value 100 accuracy: 0.700
